In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F
import torchvision.models as models
from torchvision.models import ResNet18_Weights
import torchvision.transforms as transforms
import torchvision.models as models
import json
import os
from PIL import Image
from torchvision.ops import nms
from tqdm import tqdm

In [2]:
class ResnetBackbone(nn.Module): # get C2, C3, C4, and C5
    def __init__(self, backbone='resnet18', pretrained=True):
        super(ResnetBackbone, self).__init__()

        # feature extractor
        self.backbone = models.resnet18(weights=ResNet18_Weights.DEFAULT)
        #print(self.backbone)
        self.feature_channels=[64, 128, 256, 512] # dimension for c1, c2, c3, c4 in resnet 18

        # remove last two layers of resnet18
        model_layers = list(self.backbone.children())[:-2]
        self.backbone = nn.Sequential(*model_layers)
        print(self.backbone) # view resnet18 model

        self.stem = nn.Sequential(                  #contains first residual block of resnet
            self.backbone[0],                       # conv1
            self.backbone[1],                       # batch norm
            self.backbone[2],                       # ReLU
            self.backbone[3]                        # max pooling
        )

        self.layer1 = self.backbone[4]
        self.layer2 = self.backbone[5]
        self.layer3 = self.backbone[6]
        self.layer4 = self.backbone[7]

    def forward(self, x):
        c1 = self.stem(x)
        c2 = self.layer1(c1)
        c3 = self.layer2(c2)
        c4 = self.layer3(c3)
        c5 = self.layer4(c4)
        return c2, c3, c4, c5
    
class FPN(nn.Module):
    def __init__(self, backbone, out_channel=256): # out_channel represents fixed number of features outputs for C5 - C2
        super(FPN, self).__init__()
        self.backbone = backbone

        # lateral convolution, merges C# -> M#
        self.lateral_layers = nn.ModuleList([
            nn.Conv2d(self.backbone.feature_channels[3], out_channel, kernel_size=1), # feature channel corresponds to layers in ResnetBackbone, eg. feature_channel[3] = C5
            nn.Conv2d(self.backbone.feature_channels[2], out_channel, kernel_size=1),
            nn.Conv2d(self.backbone.feature_channels[1], out_channel, kernel_size=1),
            nn.Conv2d(self.backbone.feature_channels[0], out_channel, kernel_size=1)   
        ])

        # smoothing 3x3 filters, smooths M4, M3, M2 -> P4, P3, P2
        self.smoothing_layers = nn.ModuleList([         #module list stores sub modules like Sequential but layer sequence can be changed in forward pass
            nn.Conv2d(out_channel, out_channel, kernel_size=3, padding=1),
            nn.Conv2d(out_channel, out_channel, kernel_size=3, padding=1),
            nn.Conv2d(out_channel, out_channel, kernel_size=3, padding=1),
        ])

    def forward(self, x):
        c2, c3, c4, c5 = self.backbone(x) #bottom up path from resnet backbone

        m5 = self.lateral_layers[0](c5) #C5 -> M5, no upsampling!
        m4 = self.lateral_layers[1](c4) + F.interpolate(m5, size=c4.shape[2:], mode='nearest') #include smoothing and upsampling
        m3 = self.lateral_layers[2](c3) + F.interpolate(m4, size=c3.shape[2:], mode='nearest')
        m2 = self.lateral_layers[3](c2) + F.interpolate(m3, size=c2.shape[2:], mode='nearest')


        p5 = m5
        p4 = self.smoothing_layers[0](m4)
        p3 = self.smoothing_layers[1](m3)
        p2 = self.smoothing_layers[2](m2)

        return [p2, p3, p4, p5]

In [3]:
#temp = ResnetBackbone()

In [4]:
class BBoxRegressionHead(nn.Module):
    def __init__(self, in_channels=256):
        super(BBoxRegressionHead, self).__init__()
        self.conv = nn.Conv2d(in_channels, 4, kernel_size=3, padding=1) #four outputs

    def forward(self, x):
        return self.conv(x)

In [5]:
class TextDetector(nn.Module):
    def __init__(self, backbone, fpn, regression_head):
        super(TextDetector, self).__init__()
        self._backbone = backbone
        self.fpn = fpn
        self.bbox_regression_head = regression_head

    def forward(self, x):
        feature_maps = self.fpn(x) #[p2, p3, p4, p5]

        bbox_predictions = []
        for fmap in feature_maps:
            bbox_predictions.append(self.bbox_regression_head(fmap))

        return bbox_predictions

In [6]:
backbone = ResnetBackbone(backbone='resnet18', pretrained=ResNet18_Weights)

fpn = FPN(backbone)

regression_head = BBoxRegressionHead()

model = TextDetector(backbone, fpn, regression_head)

Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Con

In [7]:
#example input
x = torch.rand(1, 3, 256, 256)  #batch size = 1, 3 channels, 256x256 image

with torch.no_grad():
    pred_bboxes = model(x)

for i, pred in enumerate(pred_bboxes):
    print(f"Feature Map {i+1} Bounding Box Predictions: {pred.shape}")


Feature Map 1 Bounding Box Predictions: torch.Size([1, 4, 64, 64])
Feature Map 2 Bounding Box Predictions: torch.Size([1, 4, 32, 32])
Feature Map 3 Bounding Box Predictions: torch.Size([1, 4, 16, 16])
Feature Map 4 Bounding Box Predictions: torch.Size([1, 4, 8, 8])


In [8]:
ann_path = "./COCO-Text/COCO_Text.json"

with open(ann_path, "r") as f:
    annotations = json.load(f)
    test = f.read()

key_to_access = "imgs"
if key_to_access in annotations:
    print(json.dumps(annotations[key_to_access]["378466"], indent=4))
else:
    print(f"Key '{key_to_access}' not found in the JSON data.")

{
    "width": 612,
    "file_name": "COCO_train2014_000000378466.jpg",
    "set": "train",
    "id": 378466,
    "height": 612
}


In [9]:
class TextDataset(Dataset):
    def __init__(self, images_path, image_ids, annotations, transform=None):
        self.images_path = images_path
        self.image_ids = image_ids
        self.labels = []
        self.transform = transform
        
        self.annotations = annotations

    def __len__(self):
        return len(self.images_path)

    def __getitem__(self, idx):
        current_id = self.image_ids[idx]
        filename = self.annotations["imgs"][str(current_id)]["file_name"]  

        image = Image.open(os.path.join(self.images_path, filename))
        #image_to_labels = self.annotations["imgToAnns"][str(current_id)]

        orig_width, orig_height = image.size

        if self.transform: # must transform image
            image = self.transform(image)


        scaled_width, scaled_height = self.transform.transforms[-2].size

        
        list_of_related_annotations = self.annotations["imgToAnns"][str(current_id)]

        # empty tensor for image with no annotation
        if not list_of_related_annotations:
            target = {
                "bboxes": torch.empty((0, 4), dtype=torch.float32),
                "bbox_ids": torch.empty((0,), dtype=torch.int64),
                "centerpoints": torch.empty((0, 2), dtype=torch.float32),
                "image_id": torch.tensor([int(current_id)], dtype=torch.int64),
            }
            return image, target

        unscaled_bbox_list = []
        annotation_id_list = []

        for element in list_of_related_annotations:
            bbox = self.annotations["anns"][str(element)]["bbox"]
            unscaled_bbox_list.append(bbox)
            annotation_id_list.append(int(element))

        scaled_bbox = []
        centers = []
        scale_factor_width = scaled_width / orig_width
        scale_factor_height = scaled_height / orig_height

        for bbox in unscaled_bbox_list:
            x_min, y_min, width, height = bbox


            x_min = x_min * scale_factor_width
            y_min = y_min * scale_factor_height
            x_max = x_min + (width * scale_factor_width)
            y_max = y_min + (height * scale_factor_height) #x_min, y_min already scaled

            scaled_bbox.append([
                x_min,
                y_min,
                x_max,
                y_max
            ])

            #centerpoints
            x_center = (x_min + x_max) / 2
            y_center = (y_min + y_max) / 2
            centers.append([x_center, y_center])

        bboxes = torch.tensor(scaled_bbox, dtype=torch.float32) # IMPORTANT USING PASCAL VOC
        bbox_ids = torch.tensor(annotation_id_list, dtype=torch.int64)
        centerpoints = torch.tensor(centers, dtype=torch.float32)

        target = {
            "bboxes": bboxes,
            "bbox_ids": bbox_ids,
            "centerpoints": centerpoints,
            "image_id": torch.tensor([int(current_id)], dtype=torch.int64)
        }
        

        return image, target

In [10]:
index = 1
images = "COCO-Text/train2014"
image_ids = list(annotations["imgs"].keys())


In [11]:

#print(image_ids[index])
filename = annotations["imgs"][str(image_ids[index])]["file_name"]
image = Image.open(os.path.join(images, filename))
#image.show()

print(filename)
#print(len(os.listdir(images)))

image_to_labels = annotations["imgToAnns"][str(image_ids[index])]
#print(image_ids[index])

temp_array = []

for element in image_to_labels:
    #print(element)
    element_annotations = annotations["anns"][str(element)]
    #print(element_annotations["bbox"])
    temp_array.append(element_annotations["bbox"])
#print(temp_array)


bbox_list = []
annotation_id_list = []

for element in image_to_labels:
    bbox = annotations["anns"][str(element)]["bbox"]
    bbox_list.append(bbox)
    annotation_id_list.append(int(element))

bboxes = torch.tensor(bbox_list, dtype=torch.float32)
bbox_ids = torch.tensor(annotation_id_list, dtype=torch.float32)

print(bboxes)


'\n\n\n\n#print(image_ids[index])\nfilename = annotations["imgs"][str(image_ids[index])]["file_name"]\nimage = Image.open(os.path.join(images, filename))\n#image.show()\n\nprint(filename)\n#print(len(os.listdir(images)))\n\nimage_to_labels = annotations["imgToAnns"][str(image_ids[index])]\n#print(image_ids[index])\n\ntemp_array = []\n\nfor element in image_to_labels:\n    #print(element)\n    element_annotations = annotations["anns"][str(element)]\n    #print(element_annotations["bbox"])\n    temp_array.append(element_annotations["bbox"])\n#print(temp_array)\n\n\nbbox_list = []\nannotation_id_list = []\n\nfor element in image_to_labels:\n    bbox = annotations["anns"][str(element)]["bbox"]\n    bbox_list.append(bbox)\n    annotation_id_list.append(int(element))\n\nbboxes = torch.tensor(bbox_list, dtype=torch.float32)\nbbox_ids = torch.tensor(annotation_id_list, dtype=torch.float32)\n\nprint(bboxes)\n'

In [12]:
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor()
])

dataset = TextDataset(images, image_ids, annotations, transform)

In [13]:
image, target = dataset[1]
#print(image)
for key, value in target.items():
    if torch.is_tensor(value):
        print(f"{key}: {value.tolist()}")
    else:
        print(f"{key}: {value}")

bboxes: [[84.94117736816406, 65.4582290649414, 95.26722717285156, 75.82785034179688], [247.14622497558594, 108.23291015625, 255.60000610351562, 134.15696716308594]]
bbox_ids: [1058591, 1058590]
centerpoints: [[90.10420227050781, 70.64303588867188], [251.37310791015625, 121.19493865966797]]
image_id: [287140]


In [14]:
def custom_collate_fn(batch):
    images, targets = zip(*batch)
    images = torch.stack(images)
    return images, targets


In [15]:
dataloader = DataLoader(dataset, batch_size=4, shuffle=True, collate_fn=custom_collate_fn)

In [16]:
for batch_idx, (data, target) in enumerate(dataloader):
    if batch_idx == 0:
        print(data)
        print(target)
        break


tensor([[[[0.8196, 0.8039, 0.8235,  ..., 0.8902, 0.8863, 0.8706],
          [0.8431, 0.8157, 0.8039,  ..., 0.8863, 0.8941, 0.8745],
          [0.8235, 0.8157, 0.8078,  ..., 0.8863, 0.8941, 0.8863],
          ...,
          [0.8941, 0.8471, 0.8784,  ..., 0.9020, 0.8941, 0.8902],
          [0.9059, 0.8863, 0.8627,  ..., 0.8941, 0.8902, 0.8824],
          [0.9059, 0.8706, 0.8941,  ..., 0.8941, 0.8902, 0.8745]],

         [[0.7294, 0.7137, 0.7333,  ..., 0.8275, 0.8235, 0.8078],
          [0.7529, 0.7255, 0.7137,  ..., 0.8235, 0.8314, 0.8118],
          [0.7333, 0.7255, 0.7176,  ..., 0.8235, 0.8314, 0.8235],
          ...,
          [0.7843, 0.7373, 0.7686,  ..., 0.8431, 0.8353, 0.8314],
          [0.8000, 0.7804, 0.7569,  ..., 0.8353, 0.8314, 0.8235],
          [0.8000, 0.7647, 0.7882,  ..., 0.8353, 0.8314, 0.8157]],

         [[0.5647, 0.5490, 0.5686,  ..., 0.7255, 0.7216, 0.7059],
          [0.5882, 0.5608, 0.5490,  ..., 0.7216, 0.7294, 0.7098],
          [0.5686, 0.5608, 0.5529,  ..., 0

In [17]:
def apply_nms(pred_bboxes, pred_scores, iou_threshold=0.5):

    keep = nms(pred_bboxes, pred_scores, iou_threshold)
    return pred_bboxes[keep], pred_scores[keep]

In [19]:
class PlaceholderLoss(nn.Module):
    def __init__(self):
        super(PlaceholderLoss, self).__init__()
        self.loss_fn = nn.SmoothL1Loss()

    def forward(self, preds, targets):

        total_loss = torch.tensor(0.0, device=preds[0].device, requires_grad=True)

        for pred, target in zip(preds, targets):
            gt_bboxes = target['bboxes']
            if gt_bboxes.numel() == 0:
                continue
            
            #flatten predictions and targets
            pred = pred.permute(0, 2, 3, 1).contiguous().view(-1, 4)
            gt_bboxes = gt_bboxes.view(-1, 4)

            #ensure tensors have the same number of boxes
            num_boxes = min(pred.size(0), gt_bboxes.size(0))
            pred = pred[:num_boxes]
            gt_bboxes = gt_bboxes[:num_boxes]


            loss = self.loss_fn(pred, gt_bboxes)
            total_loss = loss + total_loss

        return total_loss


In [20]:
model = TextDetector(backbone, fpn, regression_head)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

criterion = PlaceholderLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
def train_model(model, dataloader, criterion, optimizer, num_epochs=5):
    model.train()

    for epoch in range(num_epochs):
        epoch_loss = 0.0

        for images, targets in tqdm(dataloader, desc=f"Epoch {epoch+1}/{num_epochs}"):
            images = images.to(device)
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]


            optimizer.zero_grad()
            outputs = model(images)

            loss = criterion(outputs, targets)

            if torch.is_tensor(loss):
                epoch_loss += loss.item()
            else:
                print("loss is not a tensor. check loss function.")
                continue

            loss.backward()
            optimizer.step()

        print(f"Epoch {epoch+1} Loss: {epoch_loss:.4f}")



In [21]:
train_model(model, dataloader, criterion, optimizer, num_epochs=100)

Epoch 1/100: 100%|██████████| 5/5 [00:00<00:00, 13.74it/s]


Epoch 1 Loss: 845.1751


Epoch 2/100: 100%|██████████| 5/5 [00:00<00:00, 46.96it/s]


Epoch 2 Loss: 628.5035


Epoch 3/100: 100%|██████████| 5/5 [00:00<00:00, 45.78it/s]


...

Epoch 98 Loss: 351.2224


Epoch 99/100: 100%|██████████| 5/5 [00:00<00:00, 43.39it/s]


Epoch 99 Loss: 319.9847


Epoch 100/100: 100%|██████████| 5/5 [00:00<00:00, 48.24it/s]

Epoch 100 Loss: 350.1569


In [22]:


torch.save(model.state_dict(), "TextDetectorV1.pth")